In [1]:
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import utils.constantes as constantes

# Reinterpretação do problema

Como dito anteriormente, no algoritmo original do Needleman-Wunsch, o objetivo do *loop* de execução é preencher a matriz de similaridade (equivalente à matriz de memoização da versão recursiva) com os valores acumulados dos custos e ganhos de fazer casamentos com os elementos. Mais especificamente, a posição $[i, j]$ nessa tabela equivale ao ganho acumulado de fazer o alinhamento par-a-par das substrings $v[0:i]$ e $w[0:j]$ e o valor dessa posição depende da penalidade de *indel*, da pontuação a ser adicionada/removida em um casamento e dos valores já existentes nas posições $[i-1, j]$, $[i, j-1]$ e $[i-1, j-1]$ (por isso começamos na posiçao $[0, 0]$, afinal, fazer o alinhamento entre duas strings vazias tem custo 0). Por fim, a escolha de qual decisão tomar (*indel* em $v[i]$, *indel* em $w[j]$ ou casamento) depende do maior valor agregado possível, portanto, podemos afirmar que esse algoritmo busca encontrar a **sequência de passos** que **maximizam** a pontuação acumulada final (valor na posição $[|v|, |w|]$ na matriz).

Nessa análise, os termos "Sequência de passos" e "maximização" remetem a um outro tipo de problema: **Caminhamento ótimo em grafos**. De fato, podemos remapear as estruturas auxiliares do algoritmo original como um grafo direcionado e reinterpretar esse problema como um caminhamento nesse grafo:  

Dadas strings v e w, a matriz de similaridade é uma matriz numérica de dimensões $(|v|+1, |w|+1)$ (índices começando em 0). Podemos construir um **grafo com ($|v|+1 \times |w|+1$) nós**, um para cada elemento na matriz, ou seja, o elemento na posição `matriz_similaridade[i][j]` dará origem ao nó $(i, j)$. Visando facilitar a visualização de sua estrutura, seria conveniente dispô-los em uma posição próxima à posição original em que um elemento de índices equivalentes ficaria na tabela original.  

Cada nó está associado a um valor numérico correspondente à pontuação acumulada que é inicialmente desconhecido, com exceção do nó $(0, 0)$, que inicia com 0 (não há custo ou ganho para alinhar duas sequências vazias). Sabemos que o calculo desse valor na matriz de similaridade depende dos elementos imediatamente acima, à esquerda e na diagonal superior esquerda, logo, para cada nó $(i, j)$, devemos adicionar **3 arestas direcionadas e ponderadas**, uma direcionada para o nó $(i+1, j)$, uma para o nó $(i, j+1)$ e uma terceira apontando para $(i+1, j+1)$ (com exceção dos nós nas "bordas" inferior e direita do grafo).  

Caminhar de $(i, j)$ para $(i+1, j)$ (abaixo) equivale a fazer uma inserção em w e forçar o *match* com v (ou seja, casar v[i] com um *indel* em w). Similarmente, caminhar de $(i, j)$ para $(i, j+1)$ (à direita) equivale a fazer uma inserção em v e forçar o *match* com w (ou seja, casar w[j] com um *indel* em v), logo, o peso de todas as arestas "verticais" e "horizontais" deverão ter peso igual à penalidade de indel. O peso da aresta diagonal que leva $(i, j)$ para $(i+1, j+1)$ (diagonal inferior direita) deverá ter peso igual ao custo ou ganho de se fazer o casamento entre os elementos $v[i]$ e $w[j]$.  

A partir dessa nova estrutura, podemos redefinir o objetivo do problema como sendo encontrar o caminho de **maior custo** nesse grafo, saindo do nó fonte $f = (0, 0)$ e chegando no nó sumidouro $s = (i, j)$.

## Solução por Bellman-Ford

A solução mais comum para solucionar problemas de caminho de custo ótimo envolve aplicar o algoritmo de Dijkstra, porém, na definição acima, algumas arestas podem conter pesos negativos. Dada essa restrição, iremos implementar uma variação do algoritmo menos eficiente de bellman-Ford (pois o Bellman-Ford original, assim como o Dijkstra, resolvem o caminho de custo *mínimo*, mas desejamos encontrar o caminho de custo máximo). Nesse algoritmo, devemos exaustivamente percorrer todos os vértices e relaxar todas as arestas para gerar a distância acumulada aproximada em relação à fonte.

In [2]:
def needlema_wunsch_bellman_ford(
    v,
    w,
    matriz_pontuacao=constantes.matriz_pontuacao_blosum62,
    dicionario_indice_alfabeto=constantes.dicionario_indice_alfabeto_all_amino, 
    penalidade_indel=constantes.PENALIDADE_INDEL):
    """
    Implementação do algoritmo Needleman-Wunsch que o transforma em um problema de
    caminhamento em grafos. Dadas duas strings v e w, a estrutura do grafo é gerada
    dinamicamente e executa-se o algoritmo Bellman-Ford para encontrar o caminho de
    custo máximo entre os nós (0, 0) e (|v|, |w|)
    """
    
    len_v = len(v)
    len_w = len(w)
    
    # Inicializando o grafo com seus nós. Equivale à matriz de similaridade.
    # O valor numérico associado a cada nó corresponde à distância acumulada da fonte até aquele nó
    grafo_distancias = np.ones((len_v+1, len_w+1)) * -np.inf
    grafo_distancias[0, 0] = 0
    
    # Inicializando a matriz de predecessores para facilitar o caminhamento e a geração do alinhamento
    matriz_predecessores = [[(None, None) for _ in range(len_w+1)] for _ in range(len_v+1)]
    
    # Inicializando as arestas como tuplas (i-origem, j-origem, i-destino, j-destino, peso)
    arestas = []
    for i in range(0, len_v+1):
        for j in range(0, len_w+1):
            
            if i < len_v: # Verifica se pode ir para baixo
                arestas.append((i, j, i+1, j, -penalidade_indel)) # Aresta para baixo

            if j < len_w: # Verifica se pode ir para a direita
                arestas.append((i, j, i, j+1, -penalidade_indel)) # Aresta para a direita

            if i < len_v and j < len_w: # Verifica se pode ir para diagonal baixo-direita
                peso_casamento = matriz_pontuacao[dicionario_indice_alfabeto[v[i]], dicionario_indice_alfabeto[w[j]]]
                arestas.append((i, j, i+1, j+1, peso_casamento)) # Aresta para a diagonal baixo-direita
    #print("Len: ", len(arestas))
    
    # Itere por todos os vértices e, para cada um, processe TODAS as arestas
    for _ in range(0, len_v+1):
        for _ in range(0, len_w+1):
            for aresta in arestas:
                if grafo_distancias[aresta[0], aresta[1]] + aresta[4] > grafo_distancias[aresta[2], aresta[3]]:
                    grafo_distancias[aresta[2], aresta[3]] = grafo_distancias[aresta[0], aresta[1]] + aresta[4]
                    matriz_predecessores[aresta[2]][aresta[3]] = (aresta[0], aresta[1])
    
    return grafo_distancias, matriz_predecessores

**Nota de rodapé**: Um ponto interessante relacionado à sua implementação é que, no algoritmo original, deveríamos ter implementado um passo adicional ao final do algoritmo que verifica se há ciclos com peso acumulado negativo, porém, na seção a seguir, discutiremos por que essa etapa não é necessária.

### Funções auxiliares para printar os valores

In [3]:
def computar_mapa_posicionamento_bellman_ford(matriz_predecessores, num_rows, num_cols):
    """
    Função que computa o mapa de posicionamento do NW para a versão resolvida
    com o Bellman-Ford. O parâmetro "matriz_predecessores" deve ser uma matriz
    bidimensional onde cada elemento equivale a uma dupla (i, j) que indica
    qual o predecessor desse elemento. Na prática, esse parâmetro é uma list
    de lists com tuples.
    
    """
    
    str_mapa = ""
    for i in range(num_rows):
        for j in range(num_cols):
            
            if i == 0 and j == 0:
                str_mapa += "* "
            elif matriz_predecessores[i][j] == (i-1, j-1):
                str_mapa += "\\ "
            elif matriz_predecessores[i][j] == (i, j-1):
                str_mapa += "_ "
            elif matriz_predecessores[i][j] == (i-1, j):
                str_mapa += "| "
            else:
                str_mapa += "  " # Isso inclui -inf

        str_mapa+="\n"

    return str_mapa

In [4]:
def computar_alinhamento_bellman_ford(v, w, matriz_predecessores):
    """
    Dadas as sequências v e w originais e a matriz de predecessores gerada após
    o alinhamento de v e w com Bellman-Ford, essa função computa a representação
    do alinhamento par-a-par entre as sequências, adicionando "_" em gaps.
    """
    str_v = ""
    str_w = ""
    
    no_pred = matriz_predecessores[len(v)][len(w)]
    contador_v = len(v)
    contador_w = len(w)
    
    while no_pred != (None, None):
        if no_pred == (contador_v-1, contador_w-1):
            str_v += v[contador_v-1]
            str_w += w[contador_w-1]
            contador_v-=1
            contador_w-=1
        elif no_pred == (contador_v, contador_w-1):
            str_v += "-"
            str_w += w[contador_w-1]
            contador_w-=1
        elif no_pred == (contador_v-1, contador_w):
            str_v += v[contador_v-1]
            str_w += "-"
            contador_v-=1
        
        no_pred = matriz_predecessores[no_pred[0]][no_pred[1]]

    return str_v[::-1], str_w[::-1]

In [5]:
def executar_alinhamento_e_printar_valores(v, w):
    
    print("Alinhamento global par-a-par das strings a seguir:")
    print(v)
    print(w)
    print("----------------------")
    
    grafo_distancias, matriz_predecessores = needlema_wunsch_bellman_ford(v, w)

    print("\Grafo de distâncias (equivale á Matriz de similaridade):")
    print(grafo_distancias)

    print("\nMapa de Posicionamento")
    print(computar_mapa_posicionamento_bellman_ford(matriz_predecessores, num_rows=len(v)+1, num_cols=len(w)+1))
    print("----------------------")

    print("\nMaior Pontuação: ", grafo_distancias[len(v), len(w)])

    alin_v, alin_w = computar_alinhamento_bellman_ford(v, w, matriz_predecessores)
    print("\nAlinhamento")
    print(alin_v)
    print(alin_w)

In [6]:
executar_alinhamento_e_printar_valores(v="DRET", w="DRET")

Alinhamento global par-a-par das strings a seguir:
DRET
DRET
----------------------
\Grafo de distâncias (equivale á Matriz de similaridade):
[[ 0.  0.  0.  0.  0.]
 [ 0.  6.  6.  6.  6.]
 [ 0.  6. 11. 11. 11.]
 [ 0.  6. 11. 16. 16.]
 [ 0.  6. 11. 16. 21.]]

Mapa de Posicionamento
* _ _ _ _ 
| \ _ _ _ 
| | \ _ _ 
| | | \ _ 
| | | | \ 

----------------------

Maior Pontuação:  21.0

Alinhamento
DRET
DRET


In [7]:
executar_alinhamento_e_printar_valores(v="DRET", w="DRQT")

Alinhamento global par-a-par das strings a seguir:
DRET
DRQT
----------------------
\Grafo de distâncias (equivale á Matriz de similaridade):
[[ 0.  0.  0.  0.  0.]
 [ 0.  6.  6.  6.  6.]
 [ 0.  6. 11. 11. 11.]
 [ 0.  6. 11. 13. 13.]
 [ 0.  6. 11. 13. 18.]]

Mapa de Posicionamento
* _ _ _ _ 
| \ _ _ _ 
| | \ _ _ 
| | | \ _ 
| | | | \ 

----------------------

Maior Pontuação:  18.0

Alinhamento
DRET
DRQT


In [8]:
executar_alinhamento_e_printar_valores(v="DRNTAQLLGTDTT", w="DRQTAQAAGTTTIT")

Alinhamento global par-a-par das strings a seguir:
DRNTAQLLGTDTT
DRQTAQAAGTTTIT
----------------------
\Grafo de distâncias (equivale á Matriz de similaridade):
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.]
 [ 0.  6. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [ 0.  6. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11. 11.]
 [ 0.  6. 11. 11. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16. 16.]
 [ 0.  6. 11. 11. 16. 20. 20. 20. 20. 20. 20. 20. 20. 20. 20.]
 [ 0.  6. 11. 16. 16. 20. 25. 25. 25. 25. 25. 25. 25. 25. 25.]
 [ 0.  6. 11. 16. 16. 20. 25. 25. 25. 25. 25. 25. 25. 27. 27.]
 [ 0.  6. 11. 16. 16. 20. 25. 25. 25. 25. 25. 25. 25. 27. 27.]
 [ 0.  6. 11. 16. 16. 20. 25. 25. 25. 31. 31. 31. 31. 31. 31.]
 [ 0.  6. 11. 16. 21. 21. 25. 25. 25. 31. 36. 36. 36. 36. 36.]
 [ 0.  6. 11. 16. 21. 21. 25. 25. 25. 31. 36. 36. 36. 36. 36.]
 [ 0.  6. 11. 16. 21. 21. 25. 25. 25. 31. 36. 41. 41. 41. 41.]
 [ 0.  6. 11. 16. 21

In [9]:
executar_alinhamento_e_printar_valores(v="DRQTAKAAGTD", w="ERQLAKAAAGTD")

Alinhamento global par-a-par das strings a seguir:
DRQTAKAAGTD
ERQLAKAAAGTD
----------------------
\Grafo de distâncias (equivale á Matriz de similaridade):
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  2.  6.]
 [ 0.  2.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.  7.]
 [ 0.  2.  7. 12. 12. 12. 12. 12. 12. 12. 12. 12. 12.]
 [ 0.  2.  7. 12. 12. 12. 12. 12. 12. 12. 12. 17. 17.]
 [ 0.  2.  7. 12. 12. 16. 16. 16. 16. 16. 16. 17. 17.]
 [ 0.  2.  7. 12. 12. 16. 21. 21. 21. 21. 21. 21. 21.]
 [ 0.  2.  7. 12. 12. 16. 21. 25. 25. 25. 25. 25. 25.]
 [ 0.  2.  7. 12. 12. 16. 21. 25. 29. 29. 29. 29. 29.]
 [ 0.  2.  7. 12. 12. 16. 21. 25. 29. 29. 35. 35. 35.]
 [ 0.  2.  7. 12. 12. 16. 21. 25. 29. 29. 35. 40. 40.]
 [ 0.  2.  7. 12. 12. 16. 21. 25. 29. 29. 35. 40. 46.]]

Mapa de Posicionamento
* _ _ _ _ _ _ _ _ _ _ _ _ 
| \ _ _ _ _ _ _ _ _ _ _ \ 
| | \ _ _ _ _ _ _ _ _ _ _ 
| \ | \ _ _ _ _ _ _ _ _ _ 
| | | | | \ | \ \ \ | \ _ 
| | | | | \ _ \ \ \ _ |

Os alinhamentos gerados pelo Bellman-Ford são tão bons quanto os computados pelo Needleman-Wunsch original, tanto é que a matriz de similaridade é exatamente a mesma em ambos os algoritmos. Por outro lado, a performance daquele é muito pior do que a deste: O Bellman-Ford tem complexidade $O(|V| \times |E|)$, onde $|V|$ é a quantidade de vértices e $|E|$ é a quantidade de arestas no grafo. No problema em questão, podemos computar *a priori* a quantidade de nós e arestas em função do tamanho das strings.

Dadas duas strings v e w, como o grafo é gerado a partir da matriz de similaridade definida no Needleman-Wunsch original, teremos $|v|+1 \times |w|+1$ nós. Para cada nó, com exceção dos que se encontram nas "bordas" inferior e direita, teremos que criar 3 arestas que levam aos nós abaixo, à direita e na diagonal inferior direita, logo, teremos $3 \times (|v|+1 \times |w|+1) - 2 \times |v| - 2 \times |w| + 1$ arestas (o $-2 \times |w|$ desconta os nós inferiores, o $-2 \times |v|$ desconta os da borda à direita e o $+1$ compensa pelo nó na ponta inferior direita que foi contado duas vezes).

Portanto, a complexidade do Bellman-Ford nessa implementação é:
$$
O(|V| \times |E|) = \\
O((|v|+1 \times |w|+1) \times (3 \times (|v|+1 \times |w|+1) - 2 \times |v| - 2 \times |w| + 1)) = \\
O(|v|^{2} \times |w|^{2})
$$

que é bem superior à complexidade $O(|v| \times |w|)$ do Needleman-Wunsch. Sob uma outra perspectiva, se $|v| \approx |w|$, o Needleman-Wunsch teria complexidade quadrática enquanto que o Bellman-Ford teria complexidade polinomial à quarta potência.